# [DBA Mastery](http://google.com)
> **Just another witchcraft and wizardry site and DBA tips …**

# Upgrading a SQL Server docker container using Python

This notebook uses the Docker Python API to manage SQL Server container created in Docker, in this demo I will be upgrading a SQL Server instance from SQL Server 2017 CU15 to CU16.

These are the steps:

1. Checking SQL Server images in MCR **(optional)**  
    * **Ubuntu based images**  
    * **RHEL based images** 
2. Pull images **(optional)**
3. Check images **(optional)**
4. Create a SQL Server CU15 container **(using a local volume)**
5. Check container status
6. Connect to SQL Server CU15 container and get version
7. Restore WWI database from backup
8. Stop and upgrade SQL Server CU15 to CU16
9. Check SQL Server errorlog
10. Connect to SQL Server CU16 and get version

# 1a- Checking SQL Server images in MCR - Ubuntu (Python)



In [1]:

# Importing HTTP requests library
import requests

# Listing available images at MCR for Ubuntu
ubuntu_mcr_images = requests.get('https://mcr.microsoft.com/v2/mssql/server/tags/list/')

# Displaying HTTP GET results
print ('Ubuntu based images:')
ubuntu_mcr_images.json()

# 1b- Checking SQL Server images in MCR - RHEL (Python)

In [7]:

# Importing HTTP requests library
import requests

# Listing available images at MCR for RHEL
rhel_mcr_images = requests.get('https://mcr.microsoft.com/v2/mssql/rhel/server/tags/list/')

# Displaying HTTP GET results
print ('RHEL based images:')
rhel_mcr_images.json()

# 2- Pull images (Python)

In [2]:
# Importing docker library
import docker
client = docker.from_env()

client.images.pull('mcr.microsoft.com/mssql/server:2017-CU15-ubuntu')
print ('The SQL Server 2017 - Ubuntu CU15 image has been pulled')
client.images.pull('mcr.microsoft.com/mssql/server:2017-CU16-ubuntu')
print ('The SQL Server 2017 - Ubuntu CU16 image has been pulled')

# 3- Check images (Python)

In [2]:

# Listing local images (Docker cache)
client.images.list()


# 4- Create SQL Server CU15 container (Python)

In [3]:
# Checking if existing \ Cleaning up
try:
        container = client.containers.get('PythonSQL_CU15')
        container.stop()
        container.remove()
        print ('The existing PythonSQL_CU15 container was removed ...')
except:
        print ("The PythonSQL_CU15 container does not exists ...")

# Creating Python_CU15 container
client.containers.run(
        'mcr.microsoft.com/mssql/server:2017-CU15-ubuntu',
        detach=True,
        name="PythonSQL_CU15",
        hostname="PythonSQL_CU15",
        environment={"ACCEPT_EULA":"Y","MSSQL_SA_PASSWORD":"P1th0nR0ck$!"},
        volumes= 
        {'vlm_VersionUpgrade': {'bind': '/var/opt/mssql', 'mode': 'rw'},
        '/Users/carlos/Documents/DBA Mastery/Talks-Presentations/Containers/SQLSat912': {'bind': '/Shared', 'mode': 'rw'}},
        ports={'1433/tcp': 1401}
)
print ("The PythonSQL_CU15 container was succesfully created")

# 5- Check container status (Python)

In [3]:
# Importing docker library
import docker

# Setting context to my container
container = client.containers.get('PythonSQL_CU15')

# Checking status of PythonSQL container
print ('My container: ' + container.name + ' is in ' + container.status + ' state.')

# Checking status of all containers
client.containers.list(all=True, filters={"status":"running"})

# 6- Connect to SQL Server CU15 and get version (SQL)

In [1]:
-- Checking SQL Server instance information
-- 14.0.3162.1 = CU15
-- 14.0.3223.3 = CU16
SELECT
    SERVERPROPERTY('MachineName') AS [Computer Name],
    SERVERPROPERTY('ServerName') AS [Instance Name],
    SERVERPROPERTY('ProductVersion') AS [Product Version],
    SERVERPROPERTY('ProductUpdateLevel') AS [CU],
    RIGHT(@@version, LEN(@@version)- 3 -charindex (' ON ', @@VERSION)) [OS Version],
    SERVERPROPERTY ('Edition') AS [Edition]
FROM sys.dm_os_sys_info;

# 7- Restore WWI database from backup (SQL)

In [2]:
-- Checking existing databases
SELECT name FROM sys.databases;

-- Restore WWI backup
RESTORE DATABASE WWI FROM DISK = '/Shared/wwi.bak' WITH
MOVE 'WWI_Primary' TO '/var/opt/mssql/data/WideWorldImporters.mdf',
MOVE 'WWI_UserData' TO '/var/opt/mssql/data/WideWorldImporters_userdata.ndf',
MOVE 'WWI_Log' TO '/var/opt/mssql/data/WideWorldImporters.ldf', 
MOVE 'WWI_InMemory_Data_1' TO '/var/opt/mssql/data/WideWorldImporters_InMemory_Data_1';

-- Looking for WWI database
SELECT name FROM sys.databases;

# 8- Stop and upgrade SQL Server from CU15 to CU16 (Python)

In [1]:
# Importing docker library
import docker
client = docker.from_env()

# Setting context to my container
container = client.containers.get('PythonSQL_CU15')

# Stopping PythonSQL
container.stop()

# Creating Python_CU16 container (upgrade)
client.containers.run(
        'mcr.microsoft.com/mssql/server:2017-CU16-ubuntu',
        detach=True,
        name="PythonSQL_CU16",
        hostname="PythonSQL_CU16",
        environment={"ACCEPT_EULA":"Y","MSSQL_SA_PASSWORD":"P1th0nR0ck$!"},
        volumes= 
        {'vlm_VersionUpgrade': {'bind': '/var/opt/mssql', 'mode': 'rw'},
        '/Users/carlos/Documents/DBA Mastery/Talks-Presentations/Containers/SQLSat912': {'bind': '/Shared', 'mode': 'rw'}},
        ports={'1433/tcp': 1401}
)
print ("The PythonSQL_CU16 container was succesfully created")

# 9- Check SQL Server errorlog (Python)

In [2]:
# Setting context to my container
container = client.containers.get('PythonSQL_CU16')

# Checking the log
for line in container.logs(stream=True):
    print (line.strip())


# 10- Connect to SQL Server CU16 and get version (SQL)

In [3]:
-- Checking SQL Server instance information
-- 14.0.3162.1 = CU15
-- 14.0.3223.3 = CU16
SELECT
    SERVERPROPERTY('MachineName') AS [Computer Name],
    SERVERPROPERTY('ServerNameA') AS [Instance Name],
    SERVERPROPERTY('ProductVersion') AS [Product Version],
    SERVERPROPERTY('ProductUpdateLevel') AS [CU],
    RIGHT(@@version, LEN(@@version)- 3 -charindex (' ON ', @@VERSION)) [OS Version],
    SERVERPROPERTY ('Edition') AS [Edition];

-- Looking for WWI database
SELECT name FROM sys.databases;

# Questions or comments?

>In case you want to know more about how to use this notebook, feel free to contact me: 

* Twitter:   [@dbamastery](https://twitter.com/dbamastery)
* GitHub:    [dbamaster](https://github.com/dbamaster)
* Email:      <crobles@dbamastery.com>
* Blog:        http://dbamastery.com
